# Macropinosomes Analysis

This is a jupyter notebook that is used to analyse the macropinosomes that have been images within cells.  
The analysis runs in the following order:   
1. Analyse all z stacks within an image  
2. Analyse all time points  
3. Analyse all image regions in the .czi file



---------
### Import dependancies

In [2]:
import czifile as czi 
import os

from cellpose import core, models, io, metrics

import napari
import numpy as np

import tkinter as tk
from tkinter import filedialog

import skimage
import scipy as sci
import pandas as pd

------
### Find cells in z slice. 

In [3]:
def get_labels_for_z_pos(im_dat):
    '''This method takes in 1 z-slice from the image at a time, 
    that has already been selected for analysis, and then uses the following 
    steps to locate the cells within the image.
    1. Apply a Gaussian blur to the image, sigma = 2.
    2. Perform thresholding with the Otzu filter. 
    3. Use the "fill holes" function in python to create complete cell masks. 
    4. Convert the binary image into labels. '''
    
    # Apply a gaussian filter to image. 
    blurred_im = skimage.filters.gaussian(im_dat, sigma =2)*100
    # Use the Otzu algorithm for thresholding 
    threshold = skimage.filters.threshold_otsu(blurred_im)
    
    # binarise image 
    thresh_im = np.array(blurred_im)
    # Perform Thresholding
    thresh_im[thresh_im<threshold] = 1
    thresh_im[thresh_im>=threshold] = 0
    
    # Fill holes in bonary mask. 
    thresh_im = sci.ndimage.binary_fill_holes(thresh_im)
    # Convert binary mask to labels. 
    labels = skimage.measure.label(thresh_im)

    return(labels)

-----
### Segment cells with Cellpose

In [4]:
def segment_the_cells_cellpose(cropped_im): 
    '''This method takes in 1 z-slice from the image at a time, 
    that has already been selected for analysis, and then uses the following 
    steps to locate the cells within the image.
    1. Segment the cells using cellpose model.'''
    
    # Initalise
    im = cropped_im
    cells_cwd = os.getcwd()
    
    model_loc = cells_cwd + '/Cellpose/Models/2024_05_07_14_03_55__Abhi_cells' 
    
    # Load the cellpose model.
    model = models.CellposeModel(gpu=True, model_type=model_loc)
    
    # Cellpose segmentation of the filtered data. 
    masks, flows, styles = model.eval(im, channels=[0, 0], diameter = None)
    
    return(masks)

--------
### Work out which frames to look at

In [5]:
def Get_Frames(im_data, t):
    '''Find the frames that have something worth analysing in. 
    So far this is done byb a very simple filter of "intensity above 8000 counts.'''

    # Initalise. 
    frames_to_analyse = []
    all_labels = []

    # Loop around the image z positions.
    for z in range(im_data.shape[0]):
        # If mean intensity in image reaches the threshold.
        if np.mean(im_data[z, ...]) > 8000: 
            # Extract the single frame. 
            single_frame = np.array(im_data[t, z, ...])
            # Append the frame to a list of frames. 
            frames_to_analyse.append(z)

    return(np.array(frames_to_analyse))


------
### Filter cell masks by size. 
Keep the larger cell objects. 

In [6]:
def filter_cell_labels(labs):
    '''Use an area filter to remove cell masks that are too small.'''
    
    # Define the properties we want to look at. 
    props = {'Label', 'Area'}
    # use region_props_table to discover the cell area. 
    label_props = pd.DataFrame(skimage.measure.regionprops_table(labs, properties= props))

    # Create an arbitrary cut-off for crap. 
    cutoff = 2000

    # initalise. 
    new_labels = np.zeros([labs.shape[0], labs.shape[1]], dtype = int)
    
    counter = 1
    # For all segemented cells. 
    for i in range( label_props.shape[0]):
        mask_coords = np.where(labs == label_props['Label'].iloc[i])

        if np.min(mask_coords[0]) == 0 or np.max(mask_coords[0]) == labs.shape[0]-1: 
            continue
        elif np.min(mask_coords[1]) == 0 or np.max(mask_coords[1]) == labs.shape[1]-1:
            continue 

        else: 
            
            # If the cell is greater than the cutoff. 
            if len(mask_coords[0]) > cutoff: 
                # Save the cell mask, also rename cell masks. 
                new_labels[mask_coords[0], mask_coords[1]] = int(counter)
                
                counter = counter + 1
            
    return(new_labels)

--- 
### Use cellpose to find circles.

In [7]:
def segment_the_circles_cellpose(cropped_im, cell_labels): 
    '''This method takes in 1 z-slice from the image at a time, 
    that has already been selected for analysis, and then uses the following 
    steps to locate the cells within the image.
    1. Segment the cells using cellpose model.'''
    
    # Initalise
    im = cropped_im
    circ_cwd = os.getcwd()
    circ_model_loc = circ_cwd + '/Cellpose/Models/2024_06_05_14_36_50__Abhi_cells_circs'
    
    # Load the cellpose model.
    model = models.CellposeModel(gpu=True, model_type=circ_model_loc)
    
    cells_binary = np.array(cell_labels, dtype = int)
    
    # make a binary mask of cells 
    cells_binary[cells_binary > 0] = 1
    cells_binary[cells_binary < 0] = 0

    # Cellpose segmentation of the filtered data. 
    masks, flows, styles = model.eval(im, channels=[0, 0], diameter = None)
    
    masks = masks * cells_binary
    
    return(masks)
    

---- 
### Remove overlapping masks, keeping largest

In [8]:
def analyse_masks_and_shape(circle_masks, intensity_im):
    '''For each of the frames in the masks store variable, this method 
    cycles through all of the circle masks generated by cellpose and 
    calculates the central position and area of the mask. If the mask 
    overlaps with a mask that has already been saved, the area of the 
    two spots are compared, with the larger of the spots saved into the 
    dataframe. In addition, the mask store variabale is updated to only include 
    those masks that are included in the analysis dataframe. '''

    # Initalise
    props = {'label', 'Centroid', 'Area', 'intensity_max', 'intensity_mean'}
    circle_dataframe = pd.DataFrame()
    # Copy the masks variable to preserve the original. 
    analysis_masks = np.array(circle_masks, dtype = int)

    for j in range( circle_masks.shape[0] ):
        # get the region-props tabel for the image. 
        circ_props = pd.DataFrame( skimage.measure.regionprops_table(circle_masks[j, ...], 
                                                            intensity_image = intensity_im[j, ...], 
                                                                     properties = props) )
        circ_props['Frame'] = j
        # Get the headers for the dataframe. 
        heads = circ_props.columns.values

        # If the storage dataframe is empty. 
        if len(circle_dataframe) == 0: 
            # Add first values to the dataframe. 
            circle_dataframe = pd.concat([circle_dataframe, circ_props])
        # If there are values in the dataframe. 
        else: 
            for i in range(circ_props.shape[0]):
                # Find the circles that overlap with one another though different z frames. 
                similar_circle_x = np.where(( circle_dataframe['Centroid-0'] <= circ_props['Centroid-0'].iloc[i] + 5) & 
                                           ( circle_dataframe['Centroid-0'] >= circ_props['Centroid-0'].iloc[i] - 5 ) &
                                           ( circle_dataframe['Centroid-1'] <= circ_props['Centroid-1'].iloc[i] + 5 ) & 
                                           ( circle_dataframe['Centroid-1'] >= circ_props['Centroid-1'].iloc[i] - 5 ))[0] 
                
                dictionary = {}
                for counter in range( len(heads) ):
                    # Create a dictionary for the new values.      
                    dictionary[heads[counter]] = [circ_props[heads[counter]].iloc[i]]

                # If there is no overlap. 
                if len(similar_circle_x) == 0:
                    # Add values to dataframe. 
                    circle_dataframe = pd.concat([circle_dataframe, pd.DataFrame(dictionary)], ignore_index = True, axis = 0)

                # If there is overlap. 
                else: 
                    # If the area of the new circle is larger than the previous. 
                    if circ_props['intensity_max'].iloc[i] > circle_dataframe['intensity_max'].iloc[similar_circle_x[0]]:
                        # Replace values in the dataframe. 
                        frame = circle_dataframe['Frame'].iloc[similar_circle_x[0]]
                        mask_id = circle_dataframe['label'].iloc[similar_circle_x[0]]
                        # Update the dataframe. 
                        circle_dataframe.iloc[similar_circle_x[0]] = pd.DataFrame(dictionary)
                        # Update the masks to only include the ones used for 
                        # analysis. 
                        analysis_masks[frame, ...][analysis_masks[frame, ...] == mask_id] = 0

                    else: 
                        # Replace values in the dataframe. 
                        frame = circle_dataframe['Frame'].iloc[similar_circle_x[0]]
                        mask_id = circle_dataframe['label'].iloc[similar_circle_x[0]]
                        # Update the masks to only include the ones used for 
                        # analysis. 
                        analysis_masks[frame, ...][analysis_masks[frame, ...] == mask_id] = 0

    return(circle_dataframe, analysis_masks)

--------- 
### Rename the cell masks

In [9]:
def rename_cell_masks(cell_masks, frames_of_interest): 
    '''To aid in the analysis, this method renames the cells 
    that appear across the different time points to have the same 
    label value across all time points. '''

    # Initalise
    props = {'label', 'centroid'}
    cell_mask_labels = pd.DataFrame()
    cell_masks_var = np.array(cell_masks)
    zero_array = np.zeros(cell_masks.shape)

    # For all cell masks. 
    for i in range( cell_masks_var.shape[0] ):
        if len(np.where(frames_of_interest == i)[0]) > 0:
            # Find the central point of all the cell masks. 
            cell_props = pd.DataFrame(
                         skimage.measure.regionprops_table(
                         cell_masks_var[i, ...],            
                         properties = props ) ) 
          
            # Populate the empty cell mask label variable
            if len(cell_mask_labels) == 0: 
                cell_mask_labels = pd.concat([cell_mask_labels, cell_props])
                zero_array[i, ...] = cell_masks_var[i, ...]
            else: 
                # For all cell masks in the current frame. 
                # Initialise variable. 
                single_frame_var = np.array(cell_masks_var[i, ...], dtype = int)
                for m in range(cell_props.shape[0]):     
                    # Find out if the pixel of a previous frames 
                    # mask overlaps with the current one. 
                    overlapping_mask = np.where((cell_mask_labels['centroid-0'] >= cell_props['centroid-0'].iloc[m]-30) & 
                                               (cell_mask_labels['centroid-0'] <= cell_props['centroid-0'].iloc[m]+30) &
                                               (cell_mask_labels['centroid-1'] >= cell_props['centroid-1'].iloc[m]-30) &
                                               (cell_mask_labels['centroid-1'] <= cell_props['centroid-1'].iloc[m]+30) )[0]
                    # If there is no overlap. 
                    if len(overlapping_mask) == 0:
                        # Create a new cell ID
                        new_cell_id = np.max(cell_mask_labels['label']) + 1 
                        # rename cell mask. 
                        cell_loc = np.where(single_frame_var == cell_props['label'].iloc[m])
                        zero_array[i, cell_loc[0], cell_loc[1]] = new_cell_id
                        
                        heads = cell_props.columns.values
                    
                        temp = pd.DataFrame({heads[0]:[new_cell_id], 
                                            heads[1]:[cell_props[heads[1]].iloc[m]], 
                                            heads[2]:[cell_props[heads[2]].iloc[m]]})
                        
                        # Add mask value to dataframe. 
                        cell_mask_labels = pd.concat([cell_mask_labels, 
                                            temp], ignore_index = True)
         
                    else: 
                        new_cell_id = cell_mask_labels['label'].loc[overlapping_mask[0]]
                        # rename cell mask. 
                        cell_loc = np.where(single_frame_var == cell_props['label'].iloc[m])
                        zero_array[i, cell_loc[0], cell_loc[1]] = new_cell_id
        
                        temp = pd.DataFrame(cell_props.iloc[m])
                        # Add mask value to dataframe. 
                        cell_mask_labels = pd.concat([cell_mask_labels, 
                                                     temp.transpose()], ignore_index = True)
                
    return(zero_array, cell_mask_labels)
           

-----
### Generate the output for the analysis

In [10]:
def count_circles_in_cells(frames_of_interest, cell_masks, yel_circ_props, mag_circ_props):
    '''For each of the "frames of interest" and each time point in the
    image stack, record the number of circles in each cell in the yellow
    and magenta channels. 
    This information is recorded in the dataframes containing the 
    circles' properties in each colour channel. 
    - The results of this are that the "circle property dataframe" 
        now have the following headers. 
        
    Circle Labels | Centroid-0 | Centroid-1 | Area | intensity_max | intensity_mean | Frame | Cell ID |
    
    The last two headers are added to the dataframe, with the express intention of allowing the 
    user to trace back to where in the original image the circles created within the cell are 
    counted from, and to make sure that we can attribute a number of the circles to a particular 
    cell, even if they appear across several different z positions. 
    '''

    # Initialise
    prop = {'label'}
    yel_circ_properties['Cell ID'] = 0
    mag_circ_properties['Cell ID'] = 0
    frames_of_interest = np.array(interesting_frames)
    cell_masks = np.array(cell_masks_array)
    
    # for all the important frames. 
    for k in range( cell_masks.shape[0] ): 
        # For all the cell masks. 
        for j in range( int( np.max(cell_masks[k, ...]) ) ):

            if len(np.where(frames_of_interest == k)[0]) != 0: 
                # Cell ID 
                cell_id = j + 1
                # cell_loc in image. 
                cell_loc = np.where(cell_masks[k, ...] == cell_id)
            
                # Copy the cell masks frame 
                single_frame = np.zeros([cell_masks.shape[1], 
                                         cell_masks.shape[2]])
                # binaries so only single cell is shown
                single_frame[cell_loc[0], cell_loc[1]] = 1
            
                # Find all the circles in the cell
                yel_circs = np.array(single_frame * np.array(circ_im_yel)[k, ...], dtype = int)
                mag_circs = np.array(single_frame * np.array(circ_im_mag)[k, ...], dtype = int)
                # Cell labels 
                yel_cell_labels = pd.DataFrame(skimage.measure.regionprops_table
                                           (yel_circs, properties = prop))
                mag_circ_labels = pd.DataFrame(skimage.measure.regionprops_table
                                           (mag_circs, properties = prop))
        
                # for the number of circles present in the cell. 
                for m in range( yel_cell_labels.shape[0] ):
                    # find the specific corresponding circle to this cell mask
                    yel_df_index = np.where( (yel_circ_props['label'] == yel_cell_labels['label'].iloc[m]) &
                                               (yel_circ_props['Frame'] == k))[0]
                    # Add the cell number to the properties list. 
                    if len(yel_df_index) > 0: 
                        yel_circ_props.loc[yel_df_index, 'Cell ID'] = [cell_id]
                    
                # for the number of circles present in the cell. 
                for m in range( mag_circ_labels.shape[0] ):
                    # find the specific corresponding circle to this cell mask    
                    mag_df_index = np.where( (mag_circ_props['label'] == mag_circ_labels['label'].iloc[m]) &
                                               (mag_circ_props['Frame'] == k))[0]
                    
                    if len(mag_df_index) > 0:
                        mag_circ_props.loc[mag_df_index, 'Cell ID'] = [cell_id]

    yel_circ_props = yel_circ_props.rename(columns={'label':'Macropinosomes ID', 'Frame': 'Z position'})
    mag_circ_props = mag_circ_props.rename(columns={'label':'Macropinosomes ID', 'Frame': 'Z position'})
        
    return(yel_circ_props, mag_circ_props)

-----
### Count how many circles are overlapping between the two channel

In [11]:
def calculate_the_overlap(yel_masks, mag_masks):
    '''Calculate the number of overlapping circles from the 
    two channels within the cells.'''

    # Initalise
    overlap_circs_per_frame = []
    num_of_yel_circs = []
    num_of_mag_circs = []

    # For all z positions
    for z in range( yel_masks.shape[0] ):
        # Find the number of circle masks in the 
        # Yellow channel of the image. 
        maxi_circ_id = np.max(yel_masks[z, ...])
        # Initialise. 
        overlap_circs = 0
        # If there is more than 0 circles in the frame. 
        if maxi_circ_id != 0: 
            # For all cell masks. 
            for c in range( maxi_circ_id ):
                # Find the pixels that correlate to this 
                # circle in the other channel. 
                circ_locations = np.where(yel_masks[z, ...] == c+1)
                # If their are circles in the image frame. 
                if len(circ_locations[0]) != 0:
                    # See if the magenta circles overlap the yellow image. 
                    mag_circs = mag_masks[z, circ_locations[0], circ_locations[1]]
                    # If the magenta and yellow circle overlap
                    if np.max(mag_circs) > 0:
                        overlap_circs = overlap_circs + 1 
        # Save the number of overlapping circles in the 
        # Image frame. 
        overlap_circs_per_frame.append(overlap_circs)
        num_of_yel_circs.append(np.max(yel_masks[z, ...]))
        num_of_mag_circs.append(np.max(mag_masks[z, ...]))

    return(np.array(overlap_circs_per_frame), 
           np.array(num_of_yel_circs), 
           np.array(num_of_mag_circs))

----
### Create summary dataframe. 

In [12]:
def summary_dataframe(r, t, yellow_circ_props,
                      magenta_circ_props, file_name, 
                      num_yellow_circs, num_magenta_circs):
    ''''''

    # Initalise. 
    image_overall = pd.DataFrame()
    
    for j in range( yel_im.shape[2] ): 
        # get important info for the mean dataframe. 
        yellow_sub = yellow_circ_props[yellow_circ_props['Z position'] == j]
        yel_area_circs = yellow_sub['Area'].mean()
        yel_intenisty_circs = yellow_sub['intensity_max'].mean()
    
        magenta_sub = magenta_circ_props[magenta_circ_props['Z position'] == j]
        mag_area_circs = magenta_sub['Area'].mean()
        mag_intenisty_circs = yellow_sub['intensity_max'].mean()
    
        # Store data into a dictionary. 
        frame_data = {'File Name': [os.path.basename(file_name)], 
                 'Image Region': [r], 
                 'Time Point': [t], 
                 'Z postions': [j], 
                 'Number of Cells in Frame': [np.max(cell_mask_im[j])], 
                 'Total # Yellow macropinosomes per cell': [num_yellow_circs[j]], 
                 'Total # Magena macropinosomes per cell': [num_magenta_circs[j]], 
                 'Total # Overlapping macropinosomes': [overlapping_circles[j]], 
                 '% of Yellow macropinosomes overlapping': [100*(overlapping_circles[j] / num_yellow_circs[j])],
                 '% of Magenta macropinosomes overlapping': [100*(overlapping_circles[j] / num_magenta_circs[j])],
                 'Mean area of Yellow macropinosomes overlapping per cell': [yel_area_circs/yellow_circ_props['Cell ID'].max()], 
                 'Mean area of Magenta macropinosomes overlapping per cell': [mag_area_circs/magenta_circ_props['Cell ID'].max()],
                 'Mean fluorescent intensity value per macropinosomes per cell': [yel_intenisty_circs/yellow_circ_props['Cell ID'].max()] }
        
        # Add data into dataframe. 
        image_overall = pd.concat([image_overall, pd.DataFrame(frame_data)])

    return(image_overall.fillna(0))


In [13]:
def create_folder(folder_path):

    if os.path.exists(folder_path) == False: 
        os.mkdir(folder_path)

-----
### Read image 

**The image shape:**  
3 = Image channel  
16 = Time points.   
8 = Z planes   
1024 = x/y  
1024 = y/x  

In [15]:
def load_im_data(file_name):
    '''Load the image data from the .czi file. '''

    # Extract a single file name. 
    image_data = czi.imread(file_name)[0, 0, 0, 0, 0, ..., 0]
    
    yellow_im = image_data[:, 0, ...]
    magneta_im = image_data[:, 1, ...]

    return(yellow_im, magneta_im)

-------
### Get file name 

In [16]:
# # # Creates dialogue to ask directory
# # # Get the folder containing the image stack. 
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw() # Stops a second window opening
stack_file_name = filedialog.askopenfilename(title = 'Select Stack file')

print(stack_file_name)

N:/RCORBYN/User_Data/Current_Projects/20240411_Abhi/Raw Images/Control_2min-int_t3_sum_633-80pc.czi


In [17]:
# Inialise
cell_mask_im = []
circ_im_yel = []
circ_im_mag = []

# Create a save path
save_Folder = os.path.dirname(stack_file_name) + '/' + os.path.basename(stack_file_name)[0:-4] +'/'
create_folder(save_Folder)

# Create save folders
yel_save_folder = save_Folder + '/Yellow_channel/' 
create_folder(yel_save_folder)
mag_save_folder = save_Folder + '/Magenta_channel/'  
create_folder(mag_save_folder)

summary_file_name = os.path.dirname(stack_file_name) + '/' +os.path.basename(stack_file_name)[0:-4] + '_Summary_file.xlsx'

yel_im, mag_im = load_im_data(stack_file_name)
print(yel_im.shape, mag_im.shape)

# For all the image regions in the .czi files. 
for r in range( yel_im.shape[0] ):
    # For all time points 
    for t in range( 1):#yel_im.shape[1] ):
        interesting_frames = Get_Frames(yel_im[r, t, ...], 0)
        # Inialise
        cell_mask_im = []
        circ_im_yel = []
        circ_im_mag = []
        summary_file = pd.DataFrame()
        
        # for all the frames. 
        for i in range( yel_im.shape[2] ): 
            if len(np.where(interesting_frames == i)[0]) > 0:  
            
                single_frame_yel = yel_im[r, t, i, ...]
                single_frame_magenta = mag_im[r, t, i, ...]

                # Generate cell masks for frame i
                cell_masks = segment_the_cells_cellpose(single_frame_yel)
                # Size filter masks 
                filtered_cells = filter_cell_labels(cell_masks)
                # Add cell masks to an image
                cell_mask_im.append(filtered_cells)
                # Segment the circles using cellpose. 
                circ_mask_yel = segment_the_circles_cellpose(single_frame_yel, filtered_cells)
                # Append to a storage variable. 
                circ_im_yel.append(circ_mask_yel)
                
                # Segment the circles using cellpose. 
                circ_mask_mag = segment_the_circles_cellpose(single_frame_magenta, filtered_cells)
                # Append to a storage variable. 
                circ_im_mag.append(circ_mask_mag)
            
            else: 
                cell_mask_im.append(np.zeros(yel_im[r, t, i, ...].shape))
                circ_im_yel.append(np.zeros(yel_im[r, t, i, ...].shape))
                circ_im_mag.append(np.zeros(yel_im[r, t, i, ...].shape))

        print(r, t, i)
        
        # Rename cell masks, so that cells that appear across multiple
        # frames, have the same mask value.  
        cell_masks_array, cell_mask_props = rename_cell_masks(np.array(cell_mask_im, dtype = int), interesting_frames)
        # Convert to numpy array. 
        circ_im_yel = np.array(circ_im_yel, dtype = int) 
        circ_im_mag = np.array(circ_im_mag, dtype = int)

        # Find the "macropinosomes" (circles) that lie within the cell 
        # masks only. 
        yel_circ_properties, filtered_yel_masks = analyse_masks_and_shape(circ_im_yel, 
                                                    yel_im[r, t,  ...])
        mag_circ_properties, filtered_mag_masks = analyse_masks_and_shape(circ_im_mag, 
                                                    mag_im[r, t,  ...])
        # Update the dataframes in the lines above to contain the 
        # Frame they detected in, and the cell ID of the cell they 
        # are present in. 
        yellow_circ_props, magenta_circ_props = count_circles_in_cells(interesting_frames, 
                                                cell_masks, yel_circ_properties, mag_circ_properties)

        # find the number of overlapping macropinosomes in the two channels. 
        overlapping_circles, num_yellow_circs, num_magenta_circs = calculate_the_overlap(filtered_yel_masks, 
                                                    filtered_mag_masks)
        # Create a summary file. 
        summary = summary_dataframe(r, t, yellow_circ_props,
                              magenta_circ_props, stack_file_name, 
                              num_yellow_circs, num_magenta_circs)

        summary_file = pd.concat([summary_file, summary])

        # Save file names. 
        yel_im_name = 'Yellow_channel_' + os.path.basename(stack_file_name)[0:-4] + '_region_' + str(r) + '_timepoint_'+str(t) + '.csv'
        mag_im_name = 'Magenta_channel_' + os.path.basename(stack_file_name)[0:-4] + '_region_' + str(r) + '_timepoint_'+str(t) + '.csv'

        yellow_circ_props.to_csv(yel_save_folder + os.path.basename(yel_im_name))
        magenta_circ_props.to_csv(mag_save_folder + os.path.basename(mag_im_name))

        # Create summary file name. 
        summary_file_loc = save_Folder + os.path.basename(summary_file_name)
        # If the file needs to be created. 
        if r == 0 and t == 0: 
            # Write the summary data to the save file. 
            with pd.ExcelWriter(summary_file_loc) as writer:  
                summary_file.to_excel(writer, sheet_name='Image Region_' + str(r) + '_Time_point_' + str(t) + '.xlsx')
            # IF the file exists, append the summary file to the same file. 
        else: 
             with pd.ExcelWriter(summary_file_loc, mode='a') as writer:  
                summary_file.to_excel(writer, sheet_name='Image Region_' + str(r) + '_Time_point_' + str(t) + '.xlsx')

  

(5, 9, 5, 1024, 1024) (5, 9, 5, 1024, 1024)
0 0 0


IndexError: list index out of range

---- 
### Add the images to the viewer. 

In [ ]:
viewer1 = napari.Viewer()

viewer1.add_image(yel_im[r, t,  ...])
viewer1.add_image(mag_im[r, t,  ...])

viewer1.add_labels(np.array(filtered_yel_masks, dtype = int))
viewer1.add_labels(np.array(filtered_mag_masks, dtype = int))

In [22]:
# # Create a save path
# save_Folder = os.path.dirname(stack_file_name) + '/' + os.path.basename(stack_file_name)[0:-4] +'/'
# print(save_Folder)
# create_folder(save_Folder)

# # Create save folders
# yel_save_folder = save_Folder + '/Yellow_channel/' 
# # create_folder(yel_save_folder)
# mag_save_folder = save_Folder + '/Magenta_channel/'  
# # create_folder(mag_save_folder)

# print(yel_save_folder, mag_save_folder)

//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current Projects/20240411_Abhi/Raw Images/Control_2min-int_t3_sum_633-80pc/
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current Projects/20240411_Abhi/Raw Images/Control_2min-int_t3_sum_633-80pc//Yellow_channel/ //data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current Projects/20240411_Abhi/Raw Images/Control_2min-int_t3_sum_633-80pc//Magenta_channel/


In [18]:
import aicsimageio as aics

In [21]:
im_dat = aics.imread(stack_file_name)

In [20]:
!pip install aicspylibczi>=3.1.1

In [22]:
print(im_dat.shape)

(9, 3, 5, 1024, 1024)


In [26]:
meta_data = aics